In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib
from rdkit import Chem
from rdkit.Chem import AllChem  
import os
from torch_geometric.data import Data, DataLoader
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Performing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.inspection import permutation_importance
import shap
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib
import numbers
import torch
from torch_geometric.loader import DataLoader  

In [ ]:
import matplotlib.pyplot as plt
# plt.style.use('~/style.mplstyle')

In [ ]:
'''
'Flexural_Strength_pred’,
'Tg_pred',
'Tm_pred’,
'Tensile_Strength_pred’,
'Youngs_Modulus_pred’,
”Dielectric Constant’,
'''
Tg_xgb_model_path = "model/Tg_xgb_model.joblib"
Tg_xgb_model = joblib.load(Tg_xgb_model_path)

Tm_xgb_model_path = "model/Tm_xgb_model.joblib"
Tm_xgb_model = joblib.load(Tm_xgb_model_path)

Youngs_Modulus_xgb_model_path = "model/Youngs_Modulus_xgb_model.joblib"
Youngs_Modulus_xgb_model = joblib.load(Youngs_Modulus_xgb_model_path)

Tensile_Strength_xgb_model_path = "model/Tensile_Strength_xgb_model.joblib"
Tensile_Strength_xgb_model = joblib.load(Tensile_Strength_xgb_model_path)

Dielectric_Constant_xgb_model_path = "model/Dielectric_Constant_Total_xgb_model.joblib"
Dielectric_Constant_xgb_model = joblib.load(Dielectric_Constant_xgb_model_path)

In [ ]:
import torch

def load_data_list(file_path):
    return torch.load(file_path)

'''
'Flexural_Strength_pred’,
'Tg_pred',
'Tm_pred’,
'Tensile_Strength_pred’,
'Youngs_Modulus_pred’,
”Dielectric Constant’,
'''

dielectric_constant_load_test_dataset_path = 'experiment_dataset/test_Dielectric_Constant_Total_data_list.pt' 
Tg_load_test_dataset_path = 'experiment_dataset/test_Tg_data_list.pt'
Tm_load_test_dataset_path = 'experiment_dataset/test_Tm_data_list.pt' 
Tensile_Strength_load_test_dataset_path = 'experiment_dataset/val_Tensile_Strength_data_list.pt' 
Youngs_Modulus_load_test_dataset_path = 'experiment_dataset/val_Youngs_Modulus_data_list.pt' 

dielectric_constant_load_train_dataset_path = 'experiment_dataset/train_Dielectric_Constant_Total_data_list.pt' 
Tg_load_train_dataset_path = 'experiment_dataset/train_Tg_data_list.pt'
Tm_load_train_dataset_path = 'experiment_dataset/train_Tm_data_list.pt' 
Tensile_Strength_load_train_dataset_path = 'experiment_dataset/train_Tensile_Strength_data_list.pt' 
Youngs_Modulus_load_train_dataset_path = 'experiment_dataset/train_Youngs_Modulus_data_list.pt' 

dielectric_constant_loaded_test_data_list = load_data_list(dielectric_constant_load_test_dataset_path)
Tg_loaded_test_data_list = load_data_list(Tg_load_test_dataset_path)
Tm_loaded_test_data_list = load_data_list(Tm_load_test_dataset_path)
Tensile_Strength_loaded_test_data_list = load_data_list(Tensile_Strength_load_test_dataset_path)
Youngs_Modulus_loaded_test_data_list = load_data_list(Youngs_Modulus_load_test_dataset_path)

dielectric_constant_loaded_train_data_list = load_data_list(dielectric_constant_load_train_dataset_path)
Tg_loaded_train_data_list = load_data_list(Tg_load_train_dataset_path)
Tm_loaded_train_data_list = load_data_list(Tm_load_train_dataset_path)
Tensile_Strength_loaded_train_data_list = load_data_list(Tensile_Strength_load_train_dataset_path)
Youngs_Modulus_loaded_train_data_list = load_data_list(Youngs_Modulus_load_train_dataset_path)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def extract_features_targets(data_list, target_index, feature = "fp"):
   
    X = []
    Y = []
    for data in data_list:
        # 提取moldescriptor，假设其形状为 [1, 183]
        if feature == "2Ddescriptors":
            moldescriptor = data.moldescriptor.numpy().flatten() # 采用分子描述符作为特征
            X.append(moldescriptor)
            
        elif feature == "fp":
            fp = data.morgan_fp.numpy().flatten() # 采用分子描述符作为特征
            X.append(fp)
            
        # 提取目标y值
        target = data.y.numpy().flatten()
        Y.append(target)
        
    # print(X)
    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
'''
请你修改这段代码，新增输入值train_list，在图上同时绘制val_list与train_list的结果，图注legend放在右下角，使用不同颜色区分两个列表中的散点。
然后在text标注模型性能为测试集上的结果。
'''
def evaluate_model(model, val_list, target_index=0, figname="model_evaluation",
                   figsize=(2.3, 2.3), output=False, feature="fp", marker_style="o"):
    """
    对模型在验证集上进行评估，并打印出不同的性能指标。然后可视化预测值与真实值的散点图。
    
    参数:
        model -- 已训练的机器学习模型
        val_list -- 验证集列表（包含所有数据）
        target_index (int) -- y中要预测的索引位置，默认为0
        figname (str) -- 保存图像的文件名，不包括扩展名
        figsize (tuple) -- 图像大小，默认为 (2.3, 2.3)
        output (bool) -- 是否保存图像，默认为 False
        feature (str) -- 提取特征的类型，如 "fp"
        marker_style (str) -- 散点形状，默认为空心圆 "o"、"^"、"s"、"*" 等
    
    返回:
        mae, mse, r2 (float, float, float) -- 平均绝对误差、均方误差、决定系数
    """
    # 假设 extract_features_targets 已经定义
    X_test, y_test = extract_features_targets(val_list, target_index, feature)
    
    # 设置全局绘图参数
    plt.rcParams["font.family"] = "Arial"
    plt.rcParams["font.size"] = 7
    
    # 使用模型对测试集进行预测
    y_pred = model.predict(X_test)
    
    # 计算 MSE、MAE、RMSE、R²
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MSE (Mean Squared Error): {mse:.2f}")
    print(f"MAE (Mean Absolute Error): {mae:.2f}")
    print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
    print(f"R^2 Score: {r2:.2f}")
    
    # 绘制散点图
    fig, ax = plt.subplots(figsize=figsize)
    ax.scatter(y_pred, y_test, s=10, marker=marker_style, facecolors='none', edgecolors='grey')
    
    # 添加参考线 y = x
    ax.axline((1, 1), slope=1, color="red", linestyle="--")
    
    # 设置坐标轴标签
    ax.set_xlabel("Predicted Value")
    ax.set_ylabel("Ground Truth")
    
    # 在左上角显示 MAE、MSE、R²
    text_str = f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.2f}"
    ax.text(0.05, 0.95, text_str,
            transform=ax.transAxes,
            fontsize=7, color="black",
            verticalalignment="top", horizontalalignment="left",
            bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
    
    # 设置边框宽度
    for spine in ax.spines.values():
        spine.set_linewidth(0.75)
    
    # 保存图片（如果需要）
    if output:
        plt.savefig(f"{figname}.png", dpi=300, bbox_inches="tight")
    
    plt.show()
    return mae, mse, r2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, val_list, train_list, target_index=0, figname="model_evaluation",
                   figsize=(2.3, 2.3), output=False, feature="fp", marker_style="o"):
    """
    对模型在训练集和验证集上进行评估，并打印出测试集的性能指标。然后可视化训练集与验证集的预测值与真实值散点图。
    
    参数:
        model -- 已训练的机器学习模型
        val_list -- 验证集列表（包含所有数据）
        train_list -- 训练集列表（包含所有数据）
        target_index (int) -- y中要预测的索引位置，默认为0
        figname (str) -- 保存图像的文件名，不包括扩展名
        figsize (tuple) -- 图像大小，默认为 (2.3, 2.3)
        output (bool) -- 是否保存图像，默认为 False
        feature (str) -- 特征提取类型，如 "fp"
        marker_style (str) -- 散点形状，如 "o"、"^"、"s"、"*" 等
    
    返回:
        mae, mse, r2 (float, float, float) -- 测试集上的平均绝对误差、均方误差、决定系数
    """
    # 提取验证集特征与真实值
    X_test, y_test = extract_features_targets(val_list, target_index, feature)
    # 提取训练集特征与真实值
    X_train, y_train = extract_features_targets(train_list, target_index, feature)
    
    # 全局绘图参数设置（符合Nature期刊要求）
    plt.rcParams["font.family"] = "Arial"
    plt.rcParams["font.size"] = 7
    
    # 对训练集和验证集分别预测
    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)
    
    # 计算测试集性能指标
    mse  = mean_squared_error(y_test,  y_test_pred)
    mae  = mean_absolute_error(y_test, y_test_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_test, y_test_pred)
    
    # 打印测试集性能
    print(f"Test MSE: {mse:.2f}")
    print(f"Test MAE: {mae:.2f}")
    print(f"Test RMSE: {rmse:.2f}")
    print(f"Test R^2 Score: {r2:.2f}")
    
    # 开始绘图
    fig, ax = plt.subplots(figsize=figsize)
    # 训练集散点（蓝色空心）
    ax.scatter(y_train_pred, y_train,
               s=5, marker=marker_style,
               facecolors='none', edgecolors='#1f77b4',
               label="Train Set")
    # 验证集散点（灰色空心）
    ax.scatter(y_test_pred, y_test,
               s=5, marker=marker_style,
               facecolors='none', edgecolors='#ff7f0e',
               label="Validation Set")
    
    # 添加 y = x 参考线
    ax.axline((0, 0), slope=1, color="red", linestyle="--")
    
    # 坐标轴标签
    ax.set_xlabel("Predicted Value")
    ax.set_ylabel("Ground Truth")
    
    # 图例放右下角
    ax.legend(loc="lower right", fontsize=7)
    
    # 文本框：标注测试集性能
    text_str = (
                f"Test MAE = {mae:.2f}\n"
                f"Test MSE = {mse:.2f}\n"
                f"Test R² = {r2:.2f}")
    ax.text(0.05, 0.95, text_str,
            transform=ax.transAxes,
            fontsize=7, color="black",
            verticalalignment="top", horizontalalignment="left",
            bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
    
    # 调整边框粗细
    for spine in ax.spines.values():
        spine.set_linewidth(0.75)
    
    # 如果需要输出图片文件
    if output:
        plt.savefig(f"{figname}.png", dpi=300, bbox_inches="tight")
    
    plt.show()
    return mae, mse, r2

In [ ]:
evaluate_model(Tg_xgb_model, Tg_loaded_test_data_list, Tg_loaded_train_data_list, target_index=0, figname="Tg_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")

In [ ]:
'''
'Elongation_at_Break’,
'Tg_pred',
'Tm_pred’,
'Tensile_Strength_pred’,
'Youngs_Modulus_pred’,
”Dielectric Constant’,

target_property_col_list = ['Bandgap', 'CO2_Permeability',
       'Compressive_Strength', 'Crystallization_Temperature',
       'Elongation_at_Break', 'Flexural_Strength', 'Tg', 'H2_Permeability',
       'Hardness', 'Impact_Strength', 'Ion_Exchange_Capacity',
       'Limiting_Oxygen_Index', 'Lower_Critical_Solution_Temperature', 'Tm',
       'Methanol_Permeability', 'O2_Permeability', 'Refractive_Index',
       'Swelling_Degree', 'Thermal_Conductivity', 'Tensile_Strength', 'Td',
       'Upper_Critical_Solution_Temperature', 'Water_Contact_Angle',
       'Water_Uptake', 'Youngs_Modulus', 'Dielectric_Constant_Total'
       ]

'''
evaluate_model(Water_Uptake_xgb_model, Water_Uptake_loaded_test_data_list, Water_Uptake_loaded_train_data_list,  target_index=0, figname="Water_Uptake_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")


In [ ]:
evaluate_model(Tm_xgb_model, Tm_loaded_test_data_list, Tm_loaded_train_data_list, target_index=0, figname="Tm_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")

In [ ]:
evaluate_model(Tensile_Strength_xgb_model, Tensile_Strength_loaded_test_data_list, Tensile_Strength_loaded_train_data_list, target_index=0, figname="Tensile_Strength_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")

In [ ]:
evaluate_model(Youngs_Modulus_xgb_model, Youngs_Modulus_loaded_test_data_list, Youngs_Modulus_loaded_train_data_list, target_index=0, figname="Youngs_Modulus_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")

In [ ]:
evaluate_model(Dielectric_Constant_xgb_model, dielectric_constant_loaded_test_data_list, dielectric_constant_loaded_train_data_list, target_index=0, figname="dielectric_constant_xgboost_test",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")